In [2]:
import lightgbm as lgb

In [3]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import make_scorer

In [4]:
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sqlalchemy import create_engine
from dotenv import load_dotenv
import psycopg2

## Load data from Postgres

In [6]:
# Set postgres credentials

load_dotenv()
user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
port = os.getenv('DB_PORT')
db = os.getenv('DB')

In [7]:
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
                .format(username=user,
                        password=password,
                        ipaddress=host,
                        port=port,
                        dbname=db))
cnx = create_engine(postgres_str)

In [8]:
# List DB tables
conn = psycopg2.connect(postgres_str)
cursor = conn.cursor()
cursor.execute("""SELECT relname FROM pg_class WHERE relkind='r'
                  AND relname !~ '^(pg_|sql_)';""") # "rel" is short for relation.

tables_list = [i[0] for i in cursor.fetchall()] # A list() of tables.
tables_list #check available tables

['test',
 'shops',
 'sample_submission',
 'item_categories',
 'sales_train',
 'shops_en',
 'item_categories_en',
 'items_en',
 'items',
 'cleaning_store_id',
 'cleaning_item_category_id']

In [9]:
# Load datasets
shops_df = pd.read_sql_query('''SELECT * FROM shops_en;''', cnx)
item_categories_df = pd.read_sql_query('''SELECT * FROM item_categories_en;''', cnx)
test_df = pd.read_sql_query('''SELECT * FROM test;''', cnx)
sales_train_df = pd.read_sql_query('''SELECT * FROM sales_train;''', cnx)
items_df = pd.read_sql_query('''SELECT * FROM items_en;''', cnx)

In [10]:
items_df.info()
sales_train_df.isnull().sum() # check for NaN values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22170 entries, 0 to 22169
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   item_name         22170 non-null  object
 1   item_id           22170 non-null  int64 
 2   item_category_id  22170 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 519.7+ KB


date              0
date_block_num    0
shop_id           0
item_id           0
item_price        0
item_cnt_day      0
dtype: int64

## Preprocessing

In [11]:
# Merge with categories to get the Ids
sales_train_df = pd.merge(sales_train_df, items_df, left_on='item_id', right_on='item_id', how='left')

# Drop column that contais the item_name
sales_train_df.drop(labels='item_name', axis=1, inplace=True)

In [12]:
# Set date to YYYY/mm/dd
sales_train_df['date'] = pd.to_datetime(sales_train_df['date'], format='%d.%m.%Y')

# Drop duplicates
sales_train_df.drop_duplicates(inplace=True)

In [17]:
# Filtered item_category_id
sales_train_df = sales_train_df[~sales_train_df['item_category_id'].isin([0, 1,10,13,32,39,42,46,50,51,52,53,59,66,68,82])]

In [19]:
# Sorted date in ascending order
sales_train_df = sales_train_df.sort_values(by='date', ascending=True, na_position='first')

Starting Model 

In [24]:
# Summary Stats for each store
sales_train_df.groupby(["shop_id"]).agg({"item_cnt_day": ["count","sum", "mean", "median", "std", "min", "max"]})

item_cnt_day                                                    
               count       sum      mean median        std   min     max
shop_id                                                                 
0               9668   11435.0  1.182768    1.0   0.622336   1.0    11.0
1               5615    6137.0  1.092965    1.0   0.393995   1.0     7.0
2              25866   30460.0  1.177608    1.0   1.111459  -5.0    71.0
3              25458   28275.0  1.110653    1.0   0.992791  -2.0    83.0
4              37859   43388.0  1.146042    1.0   1.060178  -2.0   104.0
5              38080   42659.0  1.120247    1.0   0.975317  -1.0    98.0
6              81272   98696.0  1.214391    1.0   1.732674  -5.0   148.0
7              57877   66850.0  1.155036    1.0   1.007110  -2.0    67.0
8               3364    3547.0  1.054400    1.0   0.312798  -2.0     6.0
9               3718   15712.0  4.225928    2.0   8.019586  -1.0   168.0
10             21251   24289.0  1.142958    1.0   0.969262   1.0    38.0
11               499     572.0  1.146293    1.0   1.029993   1.0    21.0
12             34393   72881.0  2.119065    1.0  16.567644  -9.0  2169.0
13             17589   19435.0  1.104952    1.0   0.592782   1.0    26.0
14             36720   46084.0  1.255011    1.0   1.524763 -16.0   117.0
15             59373   71056.0  1.196773    1.0   2.024359  -1.0   255.0
16             51916   60660.0  1.168426    1.0   1.001581  -1.0    57.0
17             22873   25726.0  1.124732    1.0   0.993313  -1.0    98.0
18             52797   64947.0  1.230127    1.0   1.621742  -3.0   156.0
19             63522   72992.0  1.149082    1.0   1.060579  -2.0   103.0
20              1761    5728.0  3.252697    2.0   4.464414  -1.0    61.0
21             57364   67423.0  1.175354    1.0   1.276793  -1.0   101.0
22             45163   59815.0  1.324425    1.0   1.933671  -4.0   121.0
23              6957    7701.0  1.106943    1.0   0.604568  -1.0    23.0
24             52775   63594.0  1.205002    1.0   2.049160  -1.0   405.0
25            183303  238176.0  1.299357    1.0   2.535674  -2.0   501.0
26             53668   67611.0  1.259801    1.0   1.762897  -1.0   111.0
27            104145  134265.0  1.289212    1.0   1.675000  -2.0   151.0
28            140461  182496.0  1.299265    1.0   1.718742  -4.0   131.0
29             48756   57586.0  1.181106    1.0   1.200253  -2.0    82.0
30             50235   59801.0  1.190425    1.0   1.173373  -2.0   100.0
31            233790  308603.0  1.320001    1.0   2.203512  -2.0   288.0
32              7918    8746.0  1.104572    1.0   0.834204  -1.0    30.0
33              5018    5472.0  1.090474    1.0   0.510391  -1.0    10.0
34              5736    6432.0  1.121339    1.0   0.838019  -1.0    35.0
35             58282   68842.0  1.181188    1.0   1.236784  -2.0   102.0
36               300     324.0  1.080000    1.0   0.455659  -1.0     4.0
37             39219   45760.0  1.166781    1.0   1.153589  -1.0   100.0
38             45880   53739.0  1.171295    1.0   1.550228  -2.0   164.0
39             13377   16549.0  1.237123    1.0   1.442163  -1.0    57.0
40              4244    4928.0  1.161169    1.0   0.765716  -1.0    13.0
41             41795   49091.0  1.174566    1.0   1.104619  -3.0    62.0
42            108367  143756.0  1.326566    1.0   2.530588  -6.0   299.0
43             39162   50444.0  1.288085    1.0   2.132578  -5.0   168.0
44             38633   43158.0  1.117128    1.0   0.878386  -2.0    46.0
45             35772   41752.0  1.167170    1.0   1.228776  -2.0    82.0
46             66138   78782.0  1.191176    1.0   1.214102  -1.0    84.0
47             56534   67447.0  1.193034    1.0   1.604791  -2.0   251.0
48             21518   24810.0  1.152988    1.0   1.023958  -1.0    48.0
49             15794   17033.0  1.078448    1.0   0.489088  -1.0    21.0
50             65029   76080.0  1.169940    1.0   1.003630  -2.0    89.0
51             43906   47174.0  1.074432    1.0   0.551148   1.0  

In [26]:
sales_train_df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id
57605,2013-01-01,0,15,990,99.0,1.0,67
74711,2013-01-01,0,54,22134,399.0,2.0,40
74704,2013-01-01,0,54,22139,999.0,1.0,38
74684,2013-01-01,0,54,22151,399.0,1.0,40
48732,2013-01-01,0,15,11041,349.0,2.0,40


In [36]:
# Filtered shop_id
sales_train_df_31 = sales_train_df[sales_train_df['shop_id'] == 31]

In [37]:
sales_train_df_31

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id
65477,2013-01-02,0,31,1013,299.00,2.0,67
70951,2013-01-02,0,31,12967,199.00,1.0,55
70959,2013-01-02,0,31,12946,149.00,1.0,40
70963,2013-01-02,0,31,13735,597.43,3.0,21
70981,2013-01-02,0,31,13736,598.50,1.0,21
...,...,...,...,...,...,...,...
2890492,2015-10-31,33,31,15256,399.00,1.0,63
2926378,2015-10-31,33,31,13159,299.00,1.0,30
2890567,2015-10-31,33,31,14839,249.00,2.0,65
2890547,2015-10-31,33,31,15151,299.00,1.0,43


In [39]:
# 1. Time Related Features
#####################################################
def create_date_features(df):
    sales_train_df_31['month'] = sales_train_df_31.date.dt.month
    sales_train_df_31['day_of_month'] = sales_train_df_31.date.dt.day
    sales_train_df_31['day_of_year'] = sales_train_df_31.date.dt.dayofyear
    sales_train_df_31['week_of_year'] = sales_train_df_31.date.dt.weekofyear
    sales_train_df_31['day_of_week'] = sales_train_df_31.date.dt.dayofweek + 1
    sales_train_df_31['year'] = sales_train_df_31.date.dt.year
    sales_train_df_31["is_wknd"] = sales_train_df_31.date.dt.weekday // 4
    sales_train_df_31["quarter"] = sales_train_df_31.date.dt.quarter
    sales_train_df_31['is_month_start'] = sales_train_df_31.date.dt.is_month_start.astype(int)
    sales_train_df_31['is_month_end'] = sales_train_df_31.date.dt.is_month_end.astype(int)
    sales_train_df_31['is_quarter_start'] = sales_train_df_31.date.dt.is_quarter_start.astype(int)
    sales_train_df_31['is_quarter_end'] = sales_train_df_31.date.dt.is_quarter_end.astype(int)
    sales_train_df_31['is_year_start'] = sales_train_df_31.date.dt.is_year_start.astype(int)
    sales_train_df_31['is_year_end'] = sales_train_df_31.date.dt.is_year_end.astype(int)
    # 0: Winter - 1: Spring - 2: Summer - 3: Fall
    sales_train_df_31["season"] = np.where(sales_train_df_31.month.isin([12,1,2]), 0, 1)
    sales_train_df_31["season"] = np.where(sales_train_df_31.month.isin([6,7,8]), 2, df["season"])
    sales_train_df_31["season"] = np.where(sales_train_df_31.month.isin([9, 10, 11]), 3, df["season"])
    return df
sales_train_df_31 = create_date_features(sales_train_df_31)

In [40]:
sales_train_df_31

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id,month,day_of_month,day_of_year,week_of_year,day_of_week,year,is_wknd,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,season
65477,2013-01-02,0,31,1013,299.00,2.0,67,1,2,2,1,3,2013,0,1,0,0,0,0,0,0,0
70951,2013-01-02,0,31,12967,199.00,1.0,55,1,2,2,1,3,2013,0,1,0,0,0,0,0,0,0
70959,2013-01-02,0,31,12946,149.00,1.0,40,1,2,2,1,3,2013,0,1,0,0,0,0,0,0,0
70963,2013-01-02,0,31,13735,597.43,3.0,21,1,2,2,1,3,2013,0,1,0,0,0,0,0,0,0
70981,2013-01-02,0,31,13736,598.50,1.0,21,1,2,2,1,3,2013,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2890492,2015-10-31,33,31,15256,399.00,1.0,63,10,31,304,44,6,2015,1,4,0,1,0,0,0,0,3
2926378,2015-10-31,33,31,13159,299.00,1.0,30,10,31,304,44,6,2015,1,4,0,1,0,0,0,0,3
2890567,2015-10-31,33,31,14839,249.00,2.0,65,10,31,304,44,6,2015,1,4,0,1,0,0,0,0,3
2890547,2015-10-31,33,31,15151,299.00,1.0,43,10,31,304,44,6,2015,1,4,0,1,0,0,0,0,3


In [21]:
# Filtered shop_id
sales_train_df_31 = sales_train_df[sales_train_df['shop_id'] == 31]

In [22]:
sales_train_df_31.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id
65477,2013-01-02,0,31,1013,299.00,2.0,67
70951,2013-01-02,0,31,12967,199.00,1.0,55
70959,2013-01-02,0,31,12946,149.00,1.0,40
70963,2013-01-02,0,31,13735,597.43,3.0,21
70981,2013-01-02,0,31,13736,598.50,1.0,21


In [23]:
# Summary Stats for each store
sales_train_df_31.groupby(["shop_id"]).agg({"item_cnt_day": ["count","sum", "mean", "median", "std", "min", "max"]})

item_cnt_day                                                 
               count       sum      mean median       std  min    max
shop_id                                                              
31            233790  308603.0  1.320001    1.0  2.203512 -2.0  288.0

In [14]:
# Filtered shop_id
sales_train_df_31 = sales_train_df[sales_train_df['shop_id'] == 31]

In [15]:
# Sorted date in ascending order
sales_train_df_31 = sales_train_df_31.sort_values(by='date', ascending=True, na_position='first')

In [16]:
# Filtered item_category_id
sales_train_df_31 = sales_train_df_31[~sales_train_df['item_category_id'].isin([0, 1,10,13,32,39,42,46,50,51,52,53,59,66,68,82])]

/var/folders/jd/69j4d5lx4s9dg3b1d0bfcxx80000gp/T/ipykernel_16226/1075818176.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sales_train_df_31 = sales_train_df_31[~sales_train_df['item_category_id'].isin([0, 1,10,13,32,39,42,46,50,51,52,53,59,66,68,82])]


In [17]:
sales_train_df_31

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id
60440,2013-01-02,0,31,4248,1993.12,1.0,23
72614,2013-01-02,0,31,17241,347.11,1.0,40
71078,2013-01-02,0,31,15060,399.00,1.0,30
63528,2013-01-02,0,31,7893,1980.00,1.0,6
71610,2013-01-02,0,31,11854,199.00,1.0,63
...,...,...,...,...,...,...,...
2925574,2015-10-31,33,31,8478,349.00,1.0,43
2928312,2015-10-31,33,31,3713,1999.00,1.0,23
2890918,2015-10-31,33,31,12974,249.00,1.0,55
2928073,2015-10-31,33,31,2955,932.67,1.0,19


In [ ]:
#sales_31_group = sales_train_df_31.groupby(['date_block_num','month']).agg({'score': 'sum', 'num_attempts': 'sum'})

In [30]:
list_unique_cat = sales_train_df_31['item_category_id'].unique()

In [31]:
list_unique_cat

array([23, 40, 30,  6, 63, 22, 38, 75, 69,  5, 37,  2, 55, 45, 19, 21, 79,
       28, 25, 62, 41, 57, 15, 33, 72, 11, 14, 35, 83, 67, 70,  3, 65, 71,
       64, 43, 29, 61,  4, 73, 49, 60, 77, 56, 58, 18, 20, 12, 17,  8, 16,
       24,  7, 47, 80])

In [32]:
sales_train_df_31_cat = sales_train_df_31.set_index('item_category_id')

In [42]:
list_seq = []
for x in list_unique_cat:
    
    s = np.array(sales_train_df_31_cat.loc[x,['date_block_num','item_price','item_id','item_cnt_day']])
    
    list_seq.append(s)


Running RNN Model 

In [37]:
X = list_seq
y = 

In [ ]:
model = Sequential()
model.add(layers.SimpleRNN(units=2, activation='tanh', input_shape=(4,3)))
model.add(layers.Dense(1, activation="linear"))

# The compilation
model.compile(loss='mse', 
              optimizer='rmsprop')  # Recommended optimizer for RNNs
# The fit
model.fit(X, y,
         batch_size=16,
         epochs=10, verbose=0)

# The prediction (one per sequence/city)
model.predict(X)

In [77]:
# date_block_num: 34
# shop_id: 60
# item_id: 21807

In [ ]:
date_block_num, shop_id
0
1
2
3
4
5
6
7